# **Zonal Statitics using Earth Engine**

>

> This script will perform Zonal Statistics of multiple locations in feature collection from Image Collections in Google Earth Engine by using two diffrent methods i.e. using for loop and without using for loop


The data used here are:-
Image Collections - *Chirps daily rainfall dataset*
Feature Collections - *Districts Boundary of Budelkhand Region in India*

In [ ]:
import ee
#ee.Authenticate()
ee.Initialize()


In [ ]:
# Importing Necessary  Libraries

import pandas as pd
import datetime
import plotly.express as px

In [ ]:
# Zonal Statistics using for loop

t1 = datetime.datetime.now() 

firstDate = '2011-07-01'
lastDate = '2011-09-01'

# Calling Image Collection from EE catalogue 
rainfall = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
        .filter(ee.Filter.date(firstDate, lastDate))
print ('No of Rainfall Images: ', rainfall.size().getInfo())

# Calling Feature Collection from EE Assests
table2 = ee.FeatureCollection("users/sukkiisukant/bundelkhand_districts")

print ('No of Feature: ', table2.size().getInfo())

dates = pd.date_range(start=firstDate,end=lastDate, closed='left')
df = pd.DataFrame(list(zip(dates)),columns =['Dates'])

def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), geom, 5000)
    return image.set(dict)

listOffeat = table2.toList(table2.size());

collectionSize = listOffeat.size().getInfo()
for i in range(collectionSize):
    aoi = ee.Feature(listOffeat.get(i))
    geom = aoi.geometry()
    name = str(aoi.get('NAME_2').getInfo())
    rainMean = rainfall.map(setProperty)
    rain_list = (rainMean.aggregate_array('precipitation').getInfo())
    df[name] = rain_list 

df = df.set_index("Dates")
print(f"Processing time: {datetime.datetime.now()-t1}")

# Download output in CSV format
df.to_csv("Zonal_sts_using_for_loop.csv")

df

In [ ]:
#  Plotting the Data
fig = px.line(df)
fig.show()

In [ ]:
#  Zonal Statistics without for loop

t1 = datetime.datetime.now()

firstDate = '2010-01-01'
lastDate = '2012-01-01'

# Calling Image Collection from EE catalogue 
rainfall = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
        .filter(ee.Filter.date(firstDate, lastDate))
print ('No of Rainfall Images: ', rainfall.size().getInfo())

# Calling Feature Collection from EE Assests
table2 = ee.FeatureCollection("users/sukkiisukant/bundelkhand_districts")

print ('No of Feature: ', table2.size().getInfo())

polygons = table2
images = rainfall

def func_1(f):    
    def func_2(i):
        mean = i.reduceRegion(ee.Reducer.mean(), f.geometry(), 5000)
        return f.setMulti(mean).set({'date': i.date()})    
    return images.map(func_2) 

results = polygons.map(func_1)

results = results.flatten()

dist_list = (results.aggregate_array('NAME_2').getInfo())
rain_list = (results.aggregate_array('precipitation').getInfo())

date_list = (results.aggregate_array('date').getInfo())
date_list = [ sub['value'] for sub in date_list] 
date_list = [datetime.datetime.fromtimestamp(x // 1000).date() for x in date_list]

df = pd.DataFrame(list(zip(date_list, dist_list, rain_list)),columns =['Dates', 'Dist', 'rain'])
df = pd.pivot_table(df, index = 'Dates', columns = 'Dist', values = 'rain', aggfunc = 'mean')

print(f"Processing time: {datetime.datetime.now()-t1}")

# Download output in CSV format
df.to_csv("Zonal_sts_without_for_loop.csv")

df